In [1]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from libs.nn.conv import CnnModel
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.optimizers import SGD
import pathlib
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.image as img
import opendatasets as od
import numpy as np
from imutils import paths
import os
from libs.preprocessing import ImageToArrayPreprocessor
from libs.preprocessing import SimplePreprocessor
from libs.datasets import SimpleDatasetLoader

In [2]:
# Download the dataset
dataset_url = 'https://www.kaggle.com/datasets/ashishjangra27/face-mask-12k-images-dataset'

# Look into the data directory
images_dir = './face-mask-12k-images-dataset/Face Mask Dataset'

images_dir_path = pathlib.Path(images_dir)
if not os.path.isdir(images_dir):
    od.download(dataset_url)


In [3]:
train_dir = f'{images_dir}/Train'
test_dir = f'{images_dir}/Test'
valid_dir = f'{images_dir}/Validation'

train_imgs = list(paths.list_images(train_dir))
test_imgs = list(paths.list_images(test_dir))
valid_imgs = list(paths.list_images(valid_dir))

len(train_imgs) + len(test_imgs) + len(valid_imgs)

11792

In [4]:
train_img_paths = list(paths.list_images(train_dir))
valid_img_paths = list(paths.list_images(valid_dir))
test_img_paths = list(paths.list_images(test_dir))

In [5]:
BATCH_SIZE = 32
IMG_WIDTH = IMG_HEIGHT = 96

In [6]:
# initialize the image preprocessor
sp = SimplePreprocessor(IMG_WIDTH, IMG_HEIGHT)
iap = ImageToArrayPreprocessor()

# load the dataset from disk then scale the raw pixel intensities
# to the range [0,1]
sdl = SimpleDatasetLoader(preprocessor=[sp, iap])
trainX, trainY = sdl.load(train_img_paths, verbose=500)
trainX = trainX.astype("float") / 255.0

validX, validY = sdl.load(valid_img_paths, verbose=500)
validX = validX.astype("float") / 255.0

testX, testY = sdl.load(test_img_paths, verbose=500)
testX = testX.astype("float") / 255.0

[INFO] processed 500/10000
[INFO] processed 1000/10000
[INFO] processed 1500/10000
[INFO] processed 2000/10000
[INFO] processed 2500/10000
[INFO] processed 3000/10000
[INFO] processed 3500/10000
[INFO] processed 4000/10000
[INFO] processed 4500/10000
[INFO] processed 5000/10000
[INFO] processed 5500/10000
[INFO] processed 6000/10000
[INFO] processed 6500/10000
[INFO] processed 7000/10000
[INFO] processed 7500/10000
[INFO] processed 8000/10000
[INFO] processed 8500/10000
[INFO] processed 9000/10000
[INFO] processed 9500/10000
[INFO] processed 10000/10000
[INFO] processed 500/800
[INFO] processed 500/992


In [7]:
from sklearn.utils import shuffle
trainX, trainY = shuffle(trainX, trainY, random_state=42)
validX, validY = shuffle(validX, validY, random_state=42)

In [8]:
# convert the labels from integers to vectors
label_bin =  LabelBinarizer()
trainY =label_bin.fit_transform(trainY)
validY =label_bin.transform(validY)
testY =label_bin.transform(testY)

In [9]:
opt = SGD(learning_rate=1e-3)
model = CnnModel.build(width=IMG_WIDTH, height=IMG_HEIGHT, depth=3, classes=1)
model.compile(loss="binary_crossentropy",
              optimizer=opt,
              metrics=["accuracy"]
              )

H = model.fit(trainX,
              trainY,
              validation_data=(validX, validY),
              batch_size=BATCH_SIZE,
              epochs=20,
              verbose=1
              )


Epoch 1/20
313/313 [==============================] - 4s 6ms/step - loss: 0.6890 - accuracy: 0.7162 - val_loss: 0.6860 - val_accuracy: 0.7175
Epoch 2/20
313/313 [==============================] - 2s 5ms/step - loss: 0.6811 - accuracy: 0.8043 - val_loss: 0.6767 - val_accuracy: 0.8100
Epoch 3/20
313/313 [==============================] - 2s 5ms/step - loss: 0.6682 - accuracy: 0.8464 - val_loss: 0.6605 - val_accuracy: 0.8275
Epoch 4/20
313/313 [==============================] - 2s 5ms/step - loss: 0.6430 - accuracy: 0.8528 - val_loss: 0.6255 - val_accuracy: 0.8525
Epoch 5/20
313/313 [==============================] - 2s 5ms/step - loss: 0.5827 - accuracy: 0.8796 - val_loss: 0.5372 - val_accuracy: 0.8650
Epoch 6/20
313/313 [==============================] - 2s 5ms/step - loss: 0.4517 - accuracy: 0.8798 - val_loss: 0.3907 - val_accuracy: 0.8650
Epoch 7/20
313/313 [==============================] - 2s 5ms/step - loss: 0.3259 - accuracy: 0.8895 - val_loss: 0.3114 - val_accuracy: 0.8900
Epoch 

In [10]:
preds = model.predict(testX, batch_size=BATCH_SIZE)
preds[preds >= 0.5] = 1  #.argmax(axis=1)
preds[preds < 0.5 ] = 0
print(classification_report(testY,
                            preds,
                            target_names=["WithMask", "WithoutMask"])
      )

31/31 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

    WithMask       0.95      0.88      0.91       483
 WithoutMask       0.89      0.96      0.92       509

    accuracy                           0.92       992
   macro avg       0.92      0.92      0.92       992
weighted avg       0.92      0.92      0.92       992



In [11]:
import seaborn as sns

In [12]:
def plot_confusion_matrix(actual, predicted, labels, ds_type):
  cm = tf.math.confusion_matrix(actual, predicted)
  ax = sns.heatmap(cm, annot=True, fmt='g')
  sns.set(rc={'figure.figsize':(8, 8)})
  sns.set(font_scale=1.4)
  ax.set_title('Confusion matrix of action recognition for ' + ds_type)
  ax.set_xlabel('Predicted Action')
  ax.set_ylabel('Actual Action')
  plt.xticks(rotation=90)
  plt.yticks(rotation=0)
  ax.xaxis.set_ticklabels(labels)
  ax.yaxis.set_ticklabels(labels)

In [13]:
plot_confusion_matrix(testY.flatten(), preds.flatten(), ["WithMask", "WithoutMask"], 'Test')